# Simulation of TLS handshake

In [583]:
""" # Please remove the triple quotes in this line if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

**Client side**

The client generates a nonce of size 32 bytes. The client sends the nonce to the server along with a flag that is set to 1 if RSA will be used for key exchange, and set to zero if Diffie-Hellman will be used

In [584]:
import random
from Crypto.Random import get_random_bytes

def getClientNonce():
    # generate random nonce, size 32 bytes → 256-bit
    global nonce_client 
    nonce_client= get_random_bytes(32)    #RB
    # specify exchange method → 0=Diffie-Hellman / 1=RSA
    exchange_method_client=random.randint(0,1)
    print("Nonce-client: ",'\n'+'------------------',nonce_client,'\n')
    print("Exchange method-client:",exchange_method_client,'\n')

**Server side**

The server responds with another nonce of size 32 bytes and a flag indicating the choice of key exchange algorithm.

In [585]:
def getServerNonce():
    # generate random nonce, size 32 bytes → 256-bit
    global nonce_server 
    nonce_server= get_random_bytes(32)     #RS
    # specify exchange method → 0=Diffie-Hellman / 1=RSA
    global exchange_method_server
    exchange_method_server=random.randint(0,1)
    print("Nonce-server: ",'\n'+'------------------',nonce_server,'\n')
    if exchange_method_server==1:
        print("Exchange method-server that was chosen: RSA",'\n')
    else:
        print("Exchange method-server that was chosen: Diffie-Hellman",'\n')


**Generate RSA KEY pair**

The server randomly generates a pair of RSA public/private keys and sends to the client {public key, hash(public key)}. This constitutes the certificate.

In [586]:
from Crypto.PublicKey import RSA
def generateRSA_key():
    # generate RSA key pair
    global key 
    key= RSA.generate(2048)
    global publickey
    publickey = key.publickey()     # for later use (public key)
    pu_key=key.publickey().exportKey()
    # private key
    priv_key=key.exportKey()
    print(pu_key.decode(),'\n')
    # print(priv_key.decode())  #private key

**Key hashing-server**

In [587]:
import hashlib
from hashlib import sha256

def hashKey_server(key):
    # Hash key 
    global pk_client
    pk_client=key.public_key().export_key()
    my_sha = hashlib.sha256(pk_client)
    global hash_client 
    hash_client= int.from_bytes(my_sha.digest(),byteorder='big')
    print("Sent to client: {public key, hash(public key)} ",'\n'+'↓↓↓↓↓↓↓↓↓↓↓↓','\n')
    print("Public key :",'\n',pk_client.decode(),'\n')
    print("Generated hash :",'\n',hash_client,'\n')

**Key re-hashing-client**

Client re-hashes the public key and and compares with the hash that is sent. If they match the client proceeds to Step 5. Otherwise, the process ends here with an error message.

In [588]:
def keyReHash_client_check(pk_client,hash_client):
    # re-Hash key
    global out
    out=False
    my_sha = hashlib.sha256(pk_client)
    hash_server = int.from_bytes(my_sha.digest(),byteorder='big')
    if (hash_client==hash_server):
        out=True
        print("Hashes match, process continues → ",'\n')
    else:
        print("Hashes missmatch !")

# RSA-TLS

In case RSA was chosen, then the client will generate a pre-master secret and encrypt it with the public key of the server that was already sent over by the server.

In RSA key exchange the public key is used in the client to encrypt the pre-master secret and the private key is used in the server to decrypt it.

In [589]:
from Crypto.Cipher import PKCS1_OAEP
def PS_encrypt_RS(publickey):
    # encrypt the Pre-master secret with the public key of the server
    encryptor = PKCS1_OAEP.new(publickey)
    # generate pre-master secret
    global pre_masterSecret_client 
    pre_masterSecret_client= get_random_bytes(32)     #PS generated from the client 
    global pre_masterSecret_server 
    pre_masterSecret_server= get_random_bytes(32)     #PS generated from the server 
    encrypted = encryptor.encrypt(pre_masterSecret_client) # encrypt the PS
    print("Pre-master secret - Client: ",pre_masterSecret_client,'\n')
    print("Pre-master secret - Server: ",pre_masterSecret_server,'\n')
    print("Encrypted Pre-master secret with server public key: ",encrypted,'\n')

The server will decrypt it using his private key and validate the results

In [590]:
import hmac
import os
def gen_communicationKeys(pre_masterSecret_client,pre_masterSecret_server,nonce_client,nonce_server):
    # master secret generated at the client side (CC -Cipher client)
    def prf(secret,seed,numblocks):
        seed=seed
        output = b''
        a=hmac.new(secret,msg=seed,digestmod=hashlib.sha256).digest()
        for j in range(numblocks):
            output += hmac.new(secret,msg=a+seed,digestmod=hashlib.sha256).digest()
            a=hmac.new(secret,msg=a,digestmod=hashlib.sha256).digest()
        return output

    def master_secret(pms,client_random,server_random):
        out=prf(pms,client_random+server_random,2)
        return out[:48]

    global cipher_client
    cipher_client=prf(pre_masterSecret_client,nonce_client+nonce_server,1)
    global cipher_server
    cipher_server=prf(pre_masterSecret_server,nonce_client+nonce_server,1)
    global IC 
    IC= os.urandom(32) 
    global IS 
    IS= os.urandom(32) 
    print("Cipher Client - CC: ",'\n',cipher_client,'\n')
    print("Cipher Server - CS: ",'\n',cipher_server,'\n')
    print("Integral Key Client - IC: ",'\n',IC,'\n')
    print("Integral Key Server - IS: ",'\n',IS,'\n')

# Diffie-Hellman TLS

In [591]:
import random
def gen_PS_DH():
    p = random.randint(1, 100)
    g = random.randint(1, 100)
    a = random.randint(1, 100)
    b = random.randint(1, 100)
    A = ((pow(g, a)) % p)           
    B = ((pow(g, b)) % p)
    global pre_master_client 
    pre_master_client= ((pow(B, a)) % p)
    global pre_master_server 
    pre_master_server= ((pow(A, b)) % p)
    print("Pre-master secret - Client: ", str(pre_master_client),'\n')
    print("Pre-master secret - Server: ",str(pre_master_server),'\n')

In [592]:
def gen_communicationKeys_DH():
    MC_str=str(pre_master_client).encode()
    MS_str=str(pre_master_server).encode()
    gen_communicationKeys(MC_str,MS_str,nonce_client,nonce_server)

The client the sends the server hash(nonce_client, nonce_server, premaster secret, CC, IC). The server sends to the client hash(nonce_client, nonce_server, premaster secret, CS, IS). Each side will verify the values they received and print either a success or failure message


In [593]:
def verify_values_hash():
    sha_client = hashlib.sha256()
    sha_server = hashlib.sha256()
    # client → server
    if exchange_method_server==1: #RSA
        sha_client.update(pre_masterSecret_client)
        sha_server.update(pre_masterSecret_server)
    else: #DH
        sha_client.update(pre_master_client) 
        sha_server.update(pre_master_server)
    
    sha_client.update(nonce_client)
    sha_client.update(nonce_server)
    sha_client.update(cipher_client)
    sha_client.update(IC)
    # server → client
    sha_server.update(nonce_client)
    sha_server.update(nonce_server)
    sha_server.update(cipher_server)
    sha_server.update(IS)


In [594]:
def TLS_handshake():
    getClientNonce()
    getServerNonce()
    generateRSA_key()
    hashKey_server(key)
    keyReHash_client_check(pk_client,hash_client)
    if out==True:
        if exchange_method_server==1:
            PS_encrypt_RS(publickey)
            gen_communicationKeys(pre_masterSecret_client,pre_masterSecret_server,nonce_client,nonce_server)
        else:
            gen_PS_DH()
            gen_communicationKeys_DH()

    else:
        print('The hashes missmatch, process stopped !')

TLS_handshake()


Nonce-client:  
------------------ b'\xd4o\x8f\xb3x`\xca\x12?9d\x94\x94\xcd\xac\xc6~\x01A\xc2"i\xeap7-\xfa\xf4\xcf\xba|\r' 

Exchange method-client: 1 

Nonce-server:  
------------------ b"V\xfe\xe5L\x07\xaf\x19\xc4+\x17e\x04\x8f\xc3\x16\xf1\xf14\xc8i\xd2\xe1\xddw\x9fn\xb3\xf58'\x1a\xba" 

Exchange method-server that was chosen: RSA 

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA9AuytSu2LBFBIz2ZLjQe
F2PDQZ9zo4ToT6GDQQAa5d3AoOa062t3u/VIwnI4MyJPUUgjdd3KP9mQZ1d4tuv0
6KsevwBtO/VZkX86kCh/wULuNvfCC3HIxoEMZPVlGBKfcQcyUmhWPQmoSGQsY12z
PZ6kWqBMNAeyS0UeVGRft4GuHXEaneSDS7gwUdgdm8pqzgSKoapn30kpi3kFwACy
ynfZYB0rHIqp7NzDpbXKVL82qGnk4sHmfLgCbv4pmzLx6V4DNXKnXViXoq4k+MIr
z1TlbOn/Q4dUNt91y0gtAHo1p5jNkdZSrdm0nRGnMtPVkJ8DxwrMRHmgqbUnggQS
UQIDAQAB
-----END PUBLIC KEY----- 

Sent to client: {public key, hash(public key)}  
↓↓↓↓↓↓↓↓↓↓↓↓ 

Public key : 
 -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA9AuytSu2LBFBIz2ZLjQe
F2PDQZ9zo4ToT6GDQQAa5d3AoOa062t3u/VIwn